In [1]:
%matplotlib inline
import lightroot

In [ ]:
sample_time_frame = 0
stack = lightroot.io.get_stack(sample_time_frame)
#max_int = lightroot.io.get_max_int(sample_time_frame)

In [ ]:
#blobs=lightroot.blobs.detect(stack) does the following steps
stack = lightroot.blobs.sharpen(stack)
#lightroot.io.plotimg(sharpened)

In [ ]:
stack = lightroot.blobs.blob_labels(stack)
blobs = lightroot.blobs.blob_centroids(stack)
lightroot.io.overlay_blobs(stack,blobs)

In [2]:
n = 20
all_blobs = lightroot.process_files(n)
print("processing tracks")
tracks = lightroot.tracks_from_blobs(all_blobs, n,"./cached_data/{}.png")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
processing tracks
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


,epsilon,index,key,s,t,x,y,z
0,NaN,NaN,0,37.960506,0,81.0,479.0,13.5
1,NaN,NaN,1,17.549929,0,110.0,303.0,14.0
2,NaN,NaN,2,37.709415,0,133.0,230.5,16.5
3,NaN,NaN,3,26.248809,0,145.0,266.0,14.5
4,NaN,NaN,4,24.617067,0,149.0,319.5,13.5
5,NaN,NaN,5,30.364453,0,176.5,292.5,8.0
6,NaN,NaN,6,32.310989,0,202.0,165.0,18.0
7,NaN,NaN,7,21.748563,0,202.0,444.0,14.5
8,NaN,NaN,8,31.400637,0,221.5,369.0,26.5
9,NaN,NaN,9,22.671568,0,233.5,435.0,14.5
